# Spacecraft with reaction wheels and star tracker

Import modules and configure the notebook.

In [ ]:
# This module is part of the python standard library
import time

# These modules are part of other existing libraries
import numpy as np
import matplotlib.pyplot as plt

# This is my own script (it is an interface to the pybullet simulator)
import ae353_spacecraft

# I often go back and forth between making changes to my scripts and to
# the notebook in which they are used. One "gotcha" is that notebooks only
# import modules or scripts ONCE. Subsequent imports don't do anything, and
# in particular won't reflect any changes I've made to my scripts. To make
# sure that I'm working with the latest version of my code, I use this bit
# of magic, which forces the notebook to "reload" my script:
import importlib
importlib.reload(ae353_spacecraft)

Define and create an instance of the robot controller. By default, it applies zero torque to all wheels. Modify the `run` function - or anything else in the class definition - to make the controller behave differently.

In [ ]:
class RobotController:
    def __init__(self):
        self.dt = 0.01
        self.K = np.array([[-4.472135954999583,
  -2.6633170001054275e-15,
  -2.236067977499786,
  -10.016186942499067,
  -3.415143604229942e-15,
  -6.017091808790448],
 [4.472135954999583,
  -2.3056071713205627e-15,
  -2.236067977499785,
  10.016186942499067,
  -2.7307011801662124e-15,
  -6.017091808790448],
 [1.1572690410922678e-15,
  -3.162277660168382,
  -2.2360679774997867,
  -5.759584258290038e-16,
  -8.53423573678893,
  -6.017091808790452],
 [-6.028587380819639e-16,
  3.162277660168377,
  -2.236067977499784,
  1.0848399823472568e-16,
  8.534235736788922,
  -6.017091808790443]])
        self.L = np.array([[0.07961797804564567,
  -0.07961797804564563,
  0.08143660028688925,
  -1.4069664576386838,
  1.4069664576386816,
  -0.0814366002868892,
  0.08288958892424017,
  1.7006875052660206,
  -1.7006875052660178,
  -0.08288958892424012],
 [-0.0007622728966759749,
  1.0596610219328724,
  -0.0007796846229948388,
  1.0716941024061615,
  -0.012795353369965163,
  1.0838656444310872,
  -0.0007935957255450162,
  1.043521658776111,
  0.015377090260085532,
  1.1032039328692766],
 [-1.059661021932872,
  0.0007622728966759749,
  -1.0838656444310868,
  0.01279535336996519,
  -1.071694102406161,
  0.0007796846229948388,
  -1.1032039328692762,
  -0.01537709026008557,
  -1.0435216587761105,
  0.0007935957255450162],
 [0.05454245304134042,
  -0.054542453041340054,
  0.05578830379801371,
  -1.0034150482059945,
  1.0034150482059931,
  -0.05578830379801333,
  0.05678367653250941,
  1.2181324398803253,
  -1.2181324398803228,
  -0.056783676532509035],
 [-0.000511087993525494,
  0.987751839745206,
  -0.0005227621909250454,
  0.9959951252790432,
  -0.008754373527362767,
  1.0103139231927412,
  -0.0005320892934904143,
  0.9766955371524658,
  0.01054521459921468,
  1.028339904697188],
 [-0.9877518397452063,
  0.0005110879935259852,
  -1.0103139231927416,
  0.008754373527363336,
  -0.9959951252790437,
  0.0005227621909255478,
  -1.0283399046971882,
  -0.010545214599214292,
  -0.976695537152466,
  0.0005320892934909256]])
        self.A = np.array([[0.0, 0.0, 0.0, 1.0, -0.0, 0.0],[0.0, 0.0, 0.0, 0.0, 1.0, 0.0],[0.0, 0.0, 0.0, -0.0, 0.0, 1.0],[0.0, 0.0, 0.0, 0.0, -0.0, -0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]])
        self.C = np.array(
[[0.0, 0.0, -2.625, 0.0, 0.0, 0.0],
 [0.0, 2.625, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, -2.684959867111012, 0.0, 0.0, 0.0],
 [-0.39672994740302475, 2.625, -0.0, 0.0, 0.0, 0.0],
 [0.3967299474030241, 0.0, -2.625, 0.0, 0.0, 0.0],
 [-0.0, 2.684959867111012, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, -2.732864816051809, 0.0, 0.0, 0.0],
 [0.5321138432102653, 2.625, 0.0, 0.0, 0.0, 0.0],
 [-0.5321138432102644, 0.0, -2.625, 0.0, 0.0, 0.0],
 [0.0, 2.732864816051809, -0.0, 0.0, 0.0, 0.0]])
        self.B = np.array([[0.0, 0.0, 0.0, 0.0],[0.0, 0.0, 0.0, 0.0],[0.0, 0.0, 0.0, 0.0],[-0.052413575424878865, 0.052413575424878865, 0.0, 0.0],[0.0, 0.0,-0.052413575424878865,0.052413575424878865],[-0.03732329459238015,-0.03732329459238015,-0.03732329459238015,-0.03732329459238015]])
        self.Geq = np.array([[-0.0],
 [0.0],
 [0.3967299474030241],
 [0.0],
 [0.0],
 [0.3967299474030241],
 [-0.5321138432102644],
 [0.0],
 [-0.0],
 [-0.5321138432102644]])
        # All the equilibrium values were chosen to be 0, so they are not defined here
        self.reset()
    
    def reset(self):
        self.xhat = np.zeros((6, 1))
    
    def run(self, p, v):
        z = np.array([p]).T
        y = np.array(z - self.Geq)
        u = -self.K @ self.xhat
        self.tau = np.array([u[0,0], u[1,0], u[2,0], u[3,0]])
        xhatold = self.xhat
        for i in range(self.xhat.shape[0]):
            if np.isnan(self.xhat[i,0]):
                self.xhat[i,0] = xhatold[i,0]
        self.xhat += self.dt * (self.A @ self.xhat + self.B @ u - self.L @ (self.C@self.xhat  - y))


        return self.tau

controller = RobotController()

## Do simulation with the GUI

Create an instance of the robot simulator.

In [ ]:
robot = ae353_spacecraft.RobotSimulator(display=True, shootingstar = True)

Display a snapshot.

In [ ]:
# Get snapshot as 480 x 480 x 4 numpy array of RGBA values
rgba = robot.snapshot()

# Display snapshot
plt.figure(figsize=(9, 9))
plt.imshow(rgba)

Save the snapshot.

In [ ]:
plt.imsave('my_snapshot.png', rgba)

Run the simulation.

In [ ]:
# Restore the simulation to its initial state
robot.reset()

# Restore the controller to its initial state
controller.reset()

# Choose how long we want to run the simulation, and
# compute the corresponding number of time steps
run_time = 60.0
num_steps = int(run_time/robot.dt)

# Create a dictionary in which to store results
data = {
    't': np.empty(num_steps, dtype=float),
    'v': np.empty((4, num_steps), dtype=float),
    'p': np.empty((2 * len(robot.stars), num_steps), dtype=float),
    'tau': np.empty((4, num_steps), dtype=float),
    'rpy': np.empty((3, num_steps), dtype=float),
    'angvel': np.empty((3, num_steps), dtype=float),
    'xhat': np.empty((6, num_steps), dtype=float),
}

p = np.zeros(10) #CHANGE this when changing no. of stars
v = np.zeros(4)
# Run the simulation loop
start_time = time.time()
for step in range(num_steps):
    # Get the current time
    t = robot.dt * step
    pold = p
    vold = v
    # Get the sensor measurements
    p, v = robot.get_sensor_measurements()
    # Ensuring we don't get nan values
    for i in range(p.shape[0]):
        if np.isnan(p[i]):
            p[i] = pold[i]
    # Choose the actuator command (by running the controller)
    tau = controller.run(p, v)
    if tau.shape != (4, ):
        raise Exception(f'invalid desired torques: {tau}')
    
    # Get orientation and angular velocity as ground truth
    rpy, angvel = robot.get_rpy_and_angvel()
        
    # Log the data from this time step
    data['t'][step] = t
    data['v'][:, step] = v
    data['p'][:, step] = p
    data['tau'][:, step] = tau
    data['rpy'][:, step] = rpy
    data['angvel'][:, step] = angvel
    data['xhat'][:, step] = controller.xhat.flatten()
    
    # Send the actuator commands to robot and go forward one time step
    robot.set_actuator_commands(tau)
    robot.step(t=(start_time + (robot.dt * (step + 1))))
    
print(f'elapsed time: {time.time() - start_time:.1f} (should be {run_time})')

Plot the results.

In [ ]:
# Create a figure with subplots that all share the same x-axis
fig, (ax_r,ax_p, ax_y, ax_tau, ax_v) = plt.subplots(5, 1, figsize=(9, 12), sharex=True)

# Roll, pitch, and yaw angles
ax_r.plot(data['t'], data['rpy'][0, :], label='roll (rad)', linewidth=4)
ax_r.plot(data['t'], data['xhat'][0, :], label='roll estimate', linewidth=3)

ax_p.plot(data['t'], data['rpy'][1, :], label='pitch (rad)', linewidth=4)
ax_p.plot(data['t'], data['xhat'][1, :], label='pitch estimate', linewidth=3)

ax_y.plot(data['t'], data['rpy'][2, :], label='yaw (rad)', linewidth=4)
ax_y.plot(data['t'], data['xhat'][2, :], label='yaw estimate', linewidth=3)

ax_r.grid()
ax_r.legend(fontsize=16)
ax_r.tick_params(labelsize=14)
ax_p.grid()
ax_p.legend(fontsize=16)
ax_p.tick_params(labelsize=14)
ax_y.grid()
ax_y.legend(fontsize=16)
ax_y.tick_params(labelsize=14)

# Wheel torques
for i in range(4):
    ax_tau.plot(data['t'], data['tau'][i, :], label=f'torque {i} command (N-m)', linewidth=4)
ax_tau.plot(data['t'], np.ones_like(data['t']) * robot.tau_max, '--', label='max joint torque', linewidth=4, color='C2')
ax_tau.plot(data['t'], -np.ones_like(data['t']) * robot.tau_max, '--', linewidth=4, color='C2')
ax_tau.grid()
ax_tau.legend(fontsize=16)
ax_tau.tick_params(labelsize=14)
ax_tau.set_ylim(-1.2 * robot.tau_max, 1.2 * robot.tau_max)

# Wheel speeds
for i in range(4):
    ax_v.plot(data['t'], data['v'][i, :], label=f'wheel {i} speed (rad/s)', linewidth=4)
ax_v.plot(data['t'], np.ones_like(data['t']) * robot.v_max, '--', label='max wheel speed', linewidth=4, color='C2')
ax_v.plot(data['t'], -np.ones_like(data['t']) * robot.v_max, '--', linewidth=4, color='C2')
ax_v.grid()
ax_v.legend(fontsize=16)
ax_v.tick_params(labelsize=14)
ax_v.set_ylim(-1.2 * robot.v_max, 1.2 * robot.v_max)

# Set shared x-axis properties
ax_v.set_xlabel('time (s)', fontsize=20)
ax_v.set_xlim([data['t'][0], data['t'][-1]])

# Make the arrangement of subplots look nice
fig.tight_layout()

Plot star positions in image.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9, 9))
for i in range(len(robot.stars)):
    y = data['p'][2 * i, :]
    z = data['p'][2 * i + 1, :]
    ax.plot(-y, z, label=f'star {i}', linewidth=0.5, marker='.', markersize=10)
ax.grid()
ax.legend(fontsize=16)
ax.tick_params(labelsize=14)
ax.set_xlim(-1., 1.)
ax.set_ylim(-1., 1.)

## Do simulation without the GUI and create a video

Import what we need to create a video. You must install both [imageio](https://github.com/imageio/imageio) and [imageio-ffmpeg](https://github.com/imageio/imageio-ffmpeg) in order for this to work. You can do this from a terminal (in your `ae353-bullet` environment, for example) as follows:

```
pip install imageio
pip install imageio-ffmpeg
```

In [ ]:
import imageio

Create an instance of the robot simulator.

In [ ]:
robot = ae353_spacecraft.RobotSimulator(display=False)

Display a snapshot.

In [ ]:
# Get snapshot as 480 x 480 x 4 numpy array of RGBA values
rgba = robot.snapshot()

# Display snapshot
plt.figure(figsize=(9, 9))
plt.imshow(rgba)

Save the snapshot.

In [ ]:
plt.imsave('my_snapshot.png', rgba)

Run the simulation.

In [ ]:
# Restore the simulation to its initial state
robot.reset()

# Restore the controller to its initial state
controller.reset()

# Choose how long we want to run the simulation, and
# compute the corresponding number of time steps
run_time = 1.0
num_steps = int(run_time/robot.dt)

# Create a dictionary in which to store results
data = {
    't': np.empty(num_steps, dtype=float),
    'v': np.empty((4, num_steps), dtype=float),
    'p': np.empty((2 * len(robot.stars), num_steps), dtype=float),
    'tau': np.empty((4, num_steps), dtype=float),
    'rpy': np.empty((3, num_steps), dtype=float),
    'angvel': np.empty((3, num_steps), dtype=float),
    'xhat': np.empty((6, num_steps), dtype=float),
}

# Open video
filename = 'my_video.mp4'
fps = int(1 / robot.dt)
print(f'Creating a video with name {filename} and fps {fps}')
w = imageio.get_writer(filename,
                       format='FFMPEG',
                       mode='I',
                       fps=fps)

# Add first frame to video
rgba = robot.snapshot()
w.append_data(rgba)

# Run the simulation loop
print(f' step {0:04d} / {num_steps:04d}')
start_time = time.time()
for step in range(num_steps):
    if ((step + 1) % fps) == 0:
        print(f' step {step + 1:04d} / {num_steps:04d}')
    
    # Get the current time
    t = robot.dt * step
    
    # Get the sensor measurements
    p, v = robot.get_sensor_measurements()
    
    # Choose the actuator command (by running the controller)
    tau = controller.run(p, v)
    if tau.shape != (4, ):
        raise Exception(f'invalid desired torques: {tau}')
    
    # Get orientation and angular velocity as ground truth
    rpy, angvel = robot.get_rpy_and_angvel()
        
    # Log the data from this time step
    data['t'][step] = t
    data['v'][:, step] = v
    data['p'][:, step] = p
    data['tau'][:, step] = tau
    data['rpy'][:, step] = rpy
    data['angvel'][:, step] = angvel
    data['xhat'][:, step] = controller.xhat.flatten()
    
    # Send the actuator commands to robot and go forward one time step
    robot.set_actuator_commands(tau)
    robot.step(t=(start_time + (robot.dt * (step + 1))))
    
    # Add frame to video
    rgba = robot.snapshot()
    w.append_data(rgba)

# Close video
w.close()
print(f'elapsed time: {time.time() - start_time:.1f} (should be {run_time})')

Plot the results.

In [ ]:
# Create a figure with subplots that all share the same x-axis
fig, (ax_rpy, ax_tau, ax_v) = plt.subplots(3, 1, figsize=(9, 12), sharex=True)

# Roll, pitch, and yaw angles
ax_rpy.plot(data['t'], data['rpy'][0, :], label='roll (rad)', linewidth=4)
ax_rpy.plot(data['t'], data['rpy'][1, :], label='pitch (rad)', linewidth=4)
ax_rpy.plot(data['t'], data['rpy'][2, :], label='yaw (rad)', linewidth=4)
ax_rpy.grid()
ax_rpy.legend(fontsize=16)
ax_rpy.tick_params(labelsize=14)

# Wheel torques
for i in range(4):
    ax_tau.plot(data['t'], data['tau'][i, :], label=f'torque {i} command (N-m)', linewidth=4)
ax_tau.plot(data['t'], np.ones_like(data['t']) * robot.tau_max, '--', label='max joint torque', linewidth=4, color='C2')
ax_tau.plot(data['t'], -np.ones_like(data['t']) * robot.tau_max, '--', linewidth=4, color='C2')
ax_tau.grid()
ax_tau.legend(fontsize=16)
ax_tau.tick_params(labelsize=14)
ax_tau.set_ylim(-1.2 * robot.tau_max, 1.2 * robot.tau_max)

# Wheel speeds
for i in range(4):
    ax_v.plot(data['t'], data['v'][i, :], label=f'wheel {i} speed (rad/s)', linewidth=4)
ax_v.plot(data['t'], np.ones_like(data['t']) * robot.v_max, '--', label='max wheel speed', linewidth=4, color='C2')
ax_v.plot(data['t'], -np.ones_like(data['t']) * robot.v_max, '--', linewidth=4, color='C2')
ax_v.grid()
ax_v.legend(fontsize=16)
ax_v.tick_params(labelsize=14)
ax_v.set_ylim(-1.2 * robot.v_max, 1.2 * robot.v_max)

# Set shared x-axis properties
ax_v.set_xlabel('time (s)', fontsize=20)
ax_v.set_xlim([data['t'][0], data['t'][-1]])

# Make the arrangement of subplots look nice
fig.tight_layout()

Plot star positions in image.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9, 9))
for i in range(len(robot.stars)):
    y = data['p'][2 * i, :]
    z = data['p'][2 * i + 1, :]
    ax.plot(-y, z, label=f'star {i}', linewidth=0.5, marker='.', markersize=10)
ax.grid()
ax.legend(fontsize=16)
ax.tick_params(labelsize=14)
ax.set_xlim(-1., 1.)
ax.set_ylim(-1., 1.)